# Project 4

In this project, you will summarize and present your analysis from Projects 1-3.

### Intro: Write a problem Statement/ Specific Aim for this project

>Problem Statement: Determine if there is an association between graduate school admission and the prestige of a student's undergraduate school using data from the UCLA admissions data set.

>Specific Aim:  Using students' GRE scores, GPA, and current school rankings, we can predict if a student is admitted to the UCLA Graduate School program. We will use admissions data from past years to determine which combination of student metrics is the best predictor for admission to the program. 

### Dataset:  Write up a description of your data and any cleaning that was completed

For this project we used a dataset (provided by our instructor) with information regarding students' admission to UCLAs graduate school program.  The dataset contains four columns, three of which we can use as predictor variables: GPA, GRE score, and rank (prestige) of an applicant's undergraduate alma mater.  The fourth column, Admit, is our binary target variable.  This column is an indication of whether or not a candidate was admitted to UCLA's graduate program. Our goal with this dataset will be to identify the various factors that may influence admission into graduate school.  There are 400 observations in our dataset.  The GRE and GPA columns have two missing values (398), while Prestige has one missing value (399).  We dropped these three NaN records, so 397 records remain and are used for the analysis.  From our initial exploration of the data we are able to create the data dictionary:

Variable | Summary |Description | Type of Variable
---| ---| ---
admit |admitted to UCLA or not |0 = not admitted, 1 = admitted to UCLA | categorical
gre |Graduate Record Examination (standardize test| integers - range from 200-800| discrete
gpa | Grade Point Average |floats with precision to the hundreths - range from 0.00 - 4.00 |conintuous
prestige | ranks of current school | integars 4 to 1 (1 being highest)|ordinal catagorical

Since the prestige is a categorical variable, we used dummy variables to represent the data.  Each dummy variable column is dichotomous and there are four ranks (1-4).  When creating the dummy variables we set a baseline by dropping the variable with the most occurances.  Therefore, three additional dummy_presitige columns remain in our dataset in addition to the GRE and GPA columns.

### Demo: Provide a table that explains the data by admission status

Mean (STD) or counts by admission status for each variable 


| Not Admitted (0) | Admitted (1)
---| ---|---
GPA | mean = 3.35  | mean = 3.49
GRE |mean = 573.58 | mean = 618.57
Prestige 1 | frequency = 46% | frequency = 54%
Prestige 2 | frequency = 64% | frequency = 36%
Prestige 3 |frequency = 77% | frequency = 23%
Prestige 4 |frequency = 82% | frequency = 18%


Python Approach:

>stats=df.groupby('admit').mean()

>stats['Stat']='Mean'

>stats=stats.append(df.groupby('admit').std())

>stats.fillna('Std')

>pd.crosstab(df['admit'],[df['prestige']]).apply(lambda x: x/sum(x))


### Methods: Write up the methods used in your analysis

Load the data:
>df = pd.read_csv("../assets/admissions.csv")

Generate summary statistics:
>df.count()

>df.describe()

Identify mising data and drop null values:
>df = df.dropna()

>df.isnull().sum() --- #Check that the correct null values were dropped

Create dummy variables:
>columnsToKeep = ['admit', 'gre', 'gpa']

>data = df[columnsToKeep].join(dummyPrestige.ix[:, 'prestige_2.0':])

EDA - determine the distribution of the data with histograms or density plots using matplotlib:
>fig, axs = plt.subplots(1, 3, figsize = (15,5))

>df.gre.plot(kind="density", ax=axs[0], color = 'purple',title = 'GRE Density Plot')

>df.gpa.plot(kind="density",ax=axs[1],color = 'green', title = 'GPA Density Plot')

>df.prestige.plot(kind="density",ax=axs[2],color = 'blue',title = 'Prestige Density Plot')

>fig2, axs = plt.subplots(1, 3, figsize = (15,5))

>df.gre.plot(kind="hist", ax=axs[0],color = 'purple',title = 'GRE Histogram')

>df.gpa.plot(kind="hist",ax=axs[1],color = 'green',title = 'GPA Histogram')

>df.prestige.plot(kind="hist",ax=axs[2],color = 'blue',title = 'Prestige Histogram')


EDA - test for outliers with boxplots using matplotlib:
>fig, ax2 = plt.subplots(1, figsize = (8,5))

>ax2.set_ylim([2, 5])

>df.gpa.plot.box(title = 'GPA Boxplot')

EDA - test for colinearity with a correlation matrix:
>df.corr()

Perform logistic regression and fit the model using statsmodels (manually add intercept):
>import statsmodels.api as sm

>logModel = sm.Logit(data['admit'],train_cols_stats)

>resultFitStat = logModel.fit()

>summary = resultFitStat.summary()


Perform logistic regression and fit the model using sklearn:
>import sklearn.linear_model as lm

>logModel = lm.LogisticRegression()

>resultFit = logModel.fit(train_cols,data['admit'])

>print resultFit.coef_

>print resultFit.intercept_

>print df.admit.mean()


Calculate the accuracy score using sklearn:
>from sklearn.metrics import accuracy_score

> X = train_cols, y = df['admit']

>predicted = resultFit.predict(X)

>threshold = 0.5

>predictedClasses = (predicted > threshold).astype(int)

>print accuracy_score(y, predictedClasses)


Convert predictions to probabilities and calculate the ROC curve using sklearn:
>predictedProba1 = resultFit.predict_proba(X)

>predictedProba2 = predictedProba1[:,1]

>print predictedProba2

>from sklearn import dummy, metrics

>ax = plt.subplot(111)

>vals = metrics.roc_curve(df.admit,predictedProba2)

>ax.plot(vals[0], vals[1])

>ax.set(title='Area Under the Curve', ylabel='', xlabel='', xlim=(0, 1), ylim=(0, 1))

Calculate odds ratio using statsmodels:
> np.exp(resultFitStat.params) --- #converting into odds ratio

>params = resultFitStat.params

>conf = resultFitStat.conf_int()

>conf['OR'] = params

>conf.columns = ['2.5%', '97.5%', 'OR']

>np.exp(conf)

### Results: Write up your results

Generate summary statistics:
>The standard deviation provides some idea about the distribution of scores around the mean (average). It is used to quantify the amount of variation (or dispersion) in the dataset. A low standard deviation indicates that the data points tend to be close to the mean, while a high standard deviation indicates that the data points tend to be spread out over a wider range. Therefore, GRE would have a larger STD if the data is spread out over a wider range than GPA. This makes sense based on our exploration in the previous project where we saw that the range is (200-800) for GRE and (0.00-4.00) for GPA.

Identify mising data and drop null values:
> We identified 3 null values as mentioned above.

Create dummy variables:
> Three dummy variables remain to represenet the prestige predictor: Prestige_2.0, Prestige_3.0, Prestige_4.0

EDA - determine the distribution of the data with histograms or density plots using matplotlib:
> We used density plots and histograms to view the distribution of our data.  From these plots we see a standard bell curve for GRE and GPA, which indicates that they are both normally distributed.  We can also say that the data for these variables is fairly symmetrical since the skewness values are between (-0.5 and 0.5) and the kurtosis values are between (-2 and 2).  Prestige is an ordinal categorical variable and  categorical data is not continuous like that of a normal distribution.  Therefore the density plot doesn't make much sense here since it is not continuos.  It interesting to note that GRE has a very slight negative skew even though we saw that the mean is greater than the median.  Skewness is an indication of the dataset’s symmetry, or lack of symmetry.  A perfectly symmetrical data set and normal distributions will have a skewness of 0.  The rule of thumb...if the skewness is between -0.5 and 0.5, the data are fairly symmetrical.  If the skewness is between -1 and – 0.5 or between 0.5 and 1, the data are moderately skewed.  If the skewness is less than -1 or greater than 1, the data are highly skewed.  Kurtosis indicates how the peak and tails of a distribution differ from the normal distribution.  A normal distribution has a kurtosis of zero.  The rule of thumb... an acceptable range for kurtosis is between -2 and 2.

>Our exploration showed us that our data was close enough to a normal distribution since the the skewness values are between (-0.5 and 0.5) and the kurtosis values are between (-2 and 2) for each variable.  Therefore, we can use a model that assumes normality.  

EDA - test for outliers with boxplots using matplotlib:
> Boxplots help us to understand the distribution of the data and test for outliers based on five summary statistics: minimim, maximum, first quartile, third quartile, and median. They provide a useful way for us to visualize the range and other characteristics of responses for a large group. The GRE Boxplot shows us that the GRE mean (588) is greater than the median (580), indicating a slight skewness. We can also see that there are two outliers for GRE (300 and 200) that fall below the first quartile. The GPA Boxplot shows us that there is a slightly higher variation for GPA even though the median (3.4) is about the same as the mean (3.39). We can also see that there is one outlier for GPA (2.25) that falls below the first quartile.


EDA - test for colinearity with a correlation matrix:
>'R' is a measure of the strength of the linear relationship between two variables. If we look at the predictors in the correlation matrix (which calculates Pearson's correlation coefficient 'R') we can see that the only variables that may potentially be colinear would be GPA and GRE becuase the R value (0.382) is closest to 1.  Value of ‘R’ ranges from -1.00 to +1.00; a perfect correlation is indicated by a value of 1.00, positive or negative. A value of 0 indicates that there is no correlation between these two variables.  The correlation coefficient for GPA and GRE indicates that the two variables have a very slight positive correlation.  This suggests that students with a high GPA are likely to also score high on the GRE.  Since R is so low (0.382) we can say that the correlation is weak.


Perform logistic regression and fit the model using statsmodels (manually add intercept):
>The statsmodel output gives us an overview of the coefficients of the model, how well those coefficients fit, the overall fit quality, and several other statistical measures.  The confidence interval (CI) is the lower and upper range for the distribution of coefficients for each variable.  It gives us an idea for how robust the coefficients of the model are.  IThe confidence interval contains the parameter values that, when tested, should not be rejected with the same sample. Confidence intervals of difference parameters not containing 0 imply that there is a statistically significant difference between the populations. Therefore, the significant predictors are the prestige categories and the GPA (somewhat of a strong predictor), but GRE is not. The coefficients represent the Beta values for the fit function. From the summary printed below, we can see that there is an inverse relationship between the probability of admission and the prestige of the school. Thus, the probability of admission is higher for students who attended a top ranked prestigue_1.0 school.


Perform logistic regression and fit the model using sklearn (calculate accuracy score):
> accuracy score = 0.705289672544

>Sklearn tells us that the accuracy of the model with all features (removing one rank) is about 70%. 

Convert predictions to probabilities and calculate the ROC curve using sklearn:
>AUC (area under the curve) = 0.565439582967

>This type of graph is called a Receiver Operating Characteristic curve (or ROC curve.) It is a plot of the true positive rate against the false positive rate for the different possible thresholds.  The ROC curve demonstrates a few things for us.  The closer the curve follows the left-hand border and then the top border of the ROC space, the more accurate the test. The closer the curve comes to the 45-degree diagonal of the ROC space, the less accurate the test.


Calculate odds ratio using statsmodels:
>From the odds ratio generated above for Prestige_2.0, we can see that the odds of admission decreases 50% if a student goes to a prestige_2.0 school (as compared to the baseline prestige_1.0 school).  The odds ratio for GPA here tells us that the odds of admission are approximately 218% higher (Odds Ratio x 100%) for every one-unit increase in GPA (e.g. from 3.0 to 4.0)

### Visuals: Provide a table or visualization of these results

Dataset Summary Statistics:

<img src="../UnitProject4_e.png">

<img src="../UnitProject4_a.png">

<img src="../UnitProject4_b.png">

<img src="../UnitProject4_c.png">

Correlation Matrix:

<img src="../UnitProject4_d.png">

Logistic Regression Results using statsmodels:

<img src="../UnitProject4_StatsmodelSummary.png">

ROC curve:

<img src="../UnitProject4_ROCcurve.png">

Odds Ratio:

<img src="../UnitProject4_OddsRatio.png">

### Discussion: Write up your discussion and future steps

In this project we used logistic regression for model fitting of the UCLA admissions dataset.  We predicted the admit column using GRE, GPA, and the Prestige dummy variables (prestige_2.0, prestige_3.0 and prestige_4.0). We treated prestige_1.0 as our baseline and exclude it from our fit in order to prevent multicollinearity.  We performed logistic regression using both statsmodels and sklearn.  The advantage to using statsmodels is that it produces a nice table of the results for interpretation.  Looking at the statsmodel output we can see that there is an inverse relationship between the probability of being admitted and the prestige of a student's undergraduate school.  This means the probability of being accepted into the UCLA graduate program is higher for students who attended a top ranked undergraduate school then it is for those who attened a lower ranked undergraduate school.  

We also found this true by calculating the odds ratio.  We generated the odds ratio by taking the exponential of each of the coefficients.  From the odds ratio generated above for Prestige_2.0, we can see that the odds of admission decreases 50% if a student goes to a prestige_2.0 school as compared to the baseline prestige_1.0 school.  In handcalculating the odds ratio (using the crosstab method) we also found that the odds of admission are approximately 300% higher (Odds Ratio x 100%) for students who went to a prestige_1.0 school than they are for students who did not go to a prestige_1.0 school. I.e. for students who attended a prestige_1.0 school, the odds of being admitted into the graduate program are 3.08 times the odds of studens who did not attend a prestige_1.0 school.


Some future steps may include:
> Perfoming cross validation with k-folds (found that the accuracy is roughly 70% across all five folds. The goal is to get your accuracy score consistent accross all of your folds)

> Finding more data to train the model with, since the dataset we have is very small

> Creating additional models with different methods (e.g. decision trees) in order to find the best model
